# Migrating virtual environments to the latest Python

*Note:* it is safe to run this notebook several times. If no actions are required, then nothing will be done.

## Firstly, do you need to migrate your virtual environmments?
If you created virtual environments, they will not work with the new Python.

First, let's get the list of your virtual environments using old Python:

In [ ]:
function list_old_venvs() {
    local current_python=$(python --version | cut -d ' ' -f 2 | cut -d '.' -f 1-2)
    find /home/my_venvs/ -maxdepth 3 -mindepth 3 -type d | grep '/python3.[0-9]\+' | grep -v "/python${current_python}$" | cut -d '/' -f 4 | grep -v '_migration'
    return 0
}

function list_migrate_venvs() {
    find /home/my_venvs/ -maxdepth 3 -mindepth 3 -type d | grep '/python3.[0-9]\+' | cut -d '/' -f 4 | grep '_migration'
    return 0
}

function list_venvs() {
    local current_python=$(python --version | cut -d ' ' -f 2 | cut -d '.' -f 1-2)
    find /home/my_venvs/ -maxdepth 3 -mindepth 3 -type d | grep "/python${current_python}$" | cut -d '/' -f 4 | grep -v '_migration'
    return 0
}

echo "* Old envs:"
list_old_venvs
echo "* Envs to migrate:"
list_migrate_venvs
echo "* New envs:"
list_venvs

* If you do not have any virtual environment using Python 3.6 (*i.e.* previous cell's output has an empty "Old envs" list), then you do not need to migrate anything!

* If you no longer need any of the virtual environments, run the command in a terminal to a virtual environment:

`my_venvs_remove [venv_name]`

Once, this is done, you can come back to this notebook and migrate the remaining virtual environments to Python 3.8 by following the next instructions.

## Let's rename your old virtual environments

Run the next cell to rename you old Python 3.6 virtual environments.

In [ ]:
for venv in $(list_old_venvs); do
    new_name="/home/my_venvs/${venv}_migration"
    if [[ ! -d ${new_name} ]]; then
        echo "Renaming ${venv} -> ${new_name}"
        mv /home/my_venvs/${venv} ${new_name}
    else
        echo "Already exists: ${new_name}"
    fi
done
echo "All good!"

Let's now check the new name of the virtual environments that need to be migrated:

In [ ]:
list_migrate_venvs

## Let's create brand new virtual environments

The next cell will create a new Python virtual environment for each of your old Python virtual environment. Note that this will take some time (around 90 seconds per virtual environment).
It will also write a list of the python modules that were installed in a file called `requirements.txt` in each new virtual environments.

In [ ]:
for old_v in $(list_migrate_venvs); do
    new_v=$(echo $old_v | sed 's/_migration$//')
    
    # Loop on all venvs and create a new venv
    echo "* Creating $new_v for $old_v"
    if [[ ! -d /home/my_venvs/$new_v ]]; then
        my_venvs_create $new_v
    else
        echo "Venv ${new_v} already exists. Aborting."
        continue
    fi
    
    # Get a list of installed packages and build a requirement.txt file
    req="/home/my_venvs/${new_v}/mig_requirements.txt"
    rm -f $req
    for p in $(find /home/my_venvs/${old_v}//lib/python*/site-packages -maxdepth 1 -mindepth 1 -type d -name '*.dist-info' \
                | rev | cut -d '-' -f 3- | cut -d '/' -f 1 | rev | grep -v '^\(pip\|pkg_resources\|setuptools\|wheel\)'); do
        echo $p | tee -a $req
    done
    echo "File $req created."
    echo "Done for $new_v."
done
echo "All done!"

## Let's populate the new virtual environments with python modules

At this point, the new virtual environments are empty and we have a list of modules to install (`mig_requirements.txt`).
You can remove the old virtual environments right now, or you can do it later on.

**Run** the next cell and then copy & paste the **output text (commands)** in a terminal.

In [ ]:
for old_v in $(list_migrate_venvs); do
    new_v=$(echo $old_v | sed 's/_migration$//')
    echo "# Commands for $new_v:"
    echo "my_venvs_activate $new_v"
    echo "pip install -r /home/my_venvs/$new_v/mig_requirements.txt"
    echo "rm -f /home/my_venvs/$new_v/mig_requirements.txt"
    echo "deactivate"
    echo
done
echo "All done!"

Once you are happy with the new virtual environments, you can delete the old one (run the next cell and copy & paste the output text (commands) in a terminal).

In [ ]:
for old_v in $(list_migrate_venvs); do
    echo "# Commands for $old_v"
    echo "my_venvs_remove $old_v"
    echo
done

## Let's check your kernel spec files

Most probably, you installed a Python kernel that used to run in your old (Python 3.6) virtual environments.
We need to make sure that the kernel spec files references a Python3 executable that still exists in the new virtual environment.

**In most cases, that should be the case.** If the new virtual environment has the same name that the old one, then it should work.

In [ ]:
for kernel in $(my_kernels_list | grep '/home/.local/share/jupyter/kernels' | rev | cut -d ' ' -f 1 | rev); do
    echo "* Checking kernel spec $kernel/kernel.json:"
    cat $kernel/kernel.json | jq .
    p_exe=$(cat $kernel/kernel.json | jq ".argv[0]" | cut -d '"' -f 2)
    echo "Python executable for this kernel is $p_exe"
    if [[ ! -x $p_exe ]]; then
        echo "This Python executable is NOT correct. Please fix this kernel!"
    else
        echo "This Python executable is correct!"
    fi
    echo
done
echo "All done!"

## Manual correction of the kernel spec files

In the rare event that a kernel no longer references an existing virtual environment, execute the following steps in a terminal:

* remove the corresponding kernel (`my_kernels_remove [kernel_name]`)
* activate the virtual environment (`my_venvs_activate [virtual_env_name]`)
* install the new kernel (`my_kernels_create [kernel_name] [visible_name]`)
* deactivate the virtual environment (`deactivate`)

This will remove the old kernel spec and install a new one.

Example, from a terminal:
```
$ my_kernels_remove numpy
Kernel specs to remove:
  numpy                 /home/.local/share/jupyter/kernels/numpy
Remove 1 kernel specs [y/N]: y
[RemoveKernelSpec] Removed /home/.local/share/jupyter/kernels/numpy
$ my_venvs_activate numpy
(numpy) $ my_kernels_create numpy "P3 (numpy_venv)"
Installed kernelspec numpy in /home/.local/share/jupyter/kernels/numpy
(numpy) povalles@noto:~ $ deactivate
$
```

**End of notebook**